This tutorial and the assets can be downloaded as part of the [Wallaroo Tutorials repository](https://github.com/WallarooLabs/Wallaroo_Tutorials/blob/wallaroo2024.1_tutorials/wallaroo-run-anywhere/edge-architecture-publish-linear-regression-houseprice-model).

## Run Anywhere for ARM Architecture Tutorial: House Price Predictor Model

Wallaroo Run Anywhere provides model deployment in any device, any cloud, and any architecture.  Models uploaded to Wallaroo are set to their targeted architecture.  By default, this is `X64`.

The model's deployment configuration is tied to its architecture settings.  That deployment configuration is carried over to the models publication in Open Container Initiative (OCI) Registries, which allows edge model deployments on X64 and ARM architectures.

This tutorial demonstrates deploying a ML model trained to predict house prices to X64 and ARM edge locations through the following steps.

* Setting up a workspace, and pipeline.
* Upload a model set to the X64 architecture, and the same model set to the ARM architecture.
* Performing a sample set of inferences to verify the deployment.
* Publish the deployed model to an Open Container Initiative (OCI) Registry for both X64 and ARM deployments.
* Deploy the model to both X64 and ARM edge devices.
* Perform similar inferences on the edge devices for each architecture and show the results.

In this notebook, we use a ONNX model pre-trained to predict house prices for our examples.

## Goal

Model insights monitors the output of the house price model over a designated time window and compares it to an expected baseline distribution. We measure the performance of model deployments in different locations and compare that to the baseline to detect model drift.

### Resources

This tutorial provides the following:

* Models:
  * `models/rf_model.onnx`: The champion model trained to predict house prices.
  * Various inputs:
    * `smallinputs.df.json`: A set of house inputs that tends to generate low house price values.
    * `biginputs.df.json`: A set of house inputs that tends to generate high house price values.
    * `normal-inputs.df.json`: A set of house inputs with a range of house values.

### Prerequisites

* A deployed Wallaroo instance with [Edge Registry Services](https://docs.wallaroo.ai/wallaroo-operations-guide/wallaroo-configuration/wallaroo-edge-deployment/#enable-wallaroo-edge-deployment-registry) and [Edge Observability enabled](https://docs.wallaroo.ai/wallaroo-operations-guide/wallaroo-configuration/wallaroo-edge-deployment/#set-edge-observability-service).
* The following Python libraries installed:
  * [`wallaroo`](https://pypi.org/project/wallaroo/): The Wallaroo SDK. Included with the Wallaroo JupyterHub service by default.
  * [`pandas`](https://pypi.org/project/pandas/): Pandas, mainly used for Pandas DataFrame
  * `json`: Used for format input data for inference requests.
* A X64 Docker deployment to deploy the model on an edge location.


## Steps

* Upload the model with the targeted architecture set to `ARM`.
* Create the pipeline add the model as a model step.
* Deploy the model in the targeted architecture and perform sample inferences.
* Publish the pipeline an OCI registry.
* Deploy the model from the pipeline publish to the edge deployment with ARM architecture.
* Perform sample inferences on the ARM edge model deployment.

### Import Libraries

The first step will be to import our libraries, and set variables used through this tutorial.

In [1]:
import wallaroo
from wallaroo.object import EntityNotFoundError
from wallaroo.framework import Framework
from wallaroo.engine_config import Architecture

from IPython.display import display

# used to display DataFrame information without truncating
from IPython.display import display
import pandas as pd
pd.set_option('display.max_colwidth', None)

import datetime
import time

workspace_name = f'run-anywhere-house-price-architecture-demonstration-tutorial'
arm_pipeline_name = f'architecture-demonstration-arm'
model_name_arm = f'house-price-estimator-arm'
model_file_name = './models/rf_model.onnx'

# ignoring warnings for demonstration
import warnings
warnings.filterwarnings('ignore')

# used to display DataFrame information without truncating
from IPython.display import display
import pandas as pd
pd.set_option('display.max_colwidth', None)

In [2]:
def get_workspace(name, client):
    workspace = None
    for ws in client.list_workspaces():
        if ws.name() == name:
            workspace= ws
    if(workspace == None):
        workspace = client.create_workspace(name)
    return workspace

### Connect to the Wallaroo Instance

The first step is to connect to Wallaroo through the Wallaroo client.  The Python library is included in the Wallaroo install and available through the Jupyter Hub interface provided with your Wallaroo environment.

This is accomplished using the `wallaroo.Client()` command, which provides a URL to grant the SDK permission to your specific Wallaroo environment.  When displayed, enter the URL into a browser and confirm permissions.  Store the connection into a variable that can be referenced later.

If logging into the Wallaroo instance through the internal JupyterHub service, use `wl = wallaroo.Client()`.  For more information on Wallaroo Client settings, see the [Client Connection guide](https://docs.wallaroo.ai/wallaroo-developer-guides/wallaroo-sdk-guides/wallaroo-sdk-essentials-guide/wallaroo-sdk-essentials-client/).

In [3]:
# Login through local Wallaroo instance

wl = wallaroo.Client()

### Create Workspace

We will create a workspace to manage our pipeline and models.  The following variables will set the name of our sample workspace then set it as the current workspace.

Workspace, pipeline, and model names should be unique to each user, so we'll add in a randomly generated suffix so multiple people can run this tutorial in a Wallaroo instance without effecting each other.

In [4]:
workspace = get_workspace(workspace_name, wl)

wl.set_current_workspace(workspace)

{'name': 'run-anywhere-house-price-architecture-demonstration-tutorial', 'id': 62, 'archived': False, 'created_by': 'b4a9aa3d-83fc-407a-b4eb-37796e96f1ef', 'created_at': '2024-03-04T19:46:11.016316+00:00', 'models': [], 'pipelines': []}

## Model Deployment on ARM Architecture via the Wallaroo SDK

### Upload Models and Set ARM Target Architecture

For our example, we will upload the champion model that has been trained to derive house prices from a variety of inputs.  The model file is `rf_model.onnx`, and is uploaded with the name `house-price-estimator`.

Models are uploaded to Wallaroo via the [`wallaroo.client.upload_model`](https://docs.wallaroo.ai/wallaroo-developer-guides/wallaroo-sdk-guides/wallaroo-sdk-reference-guide/client/#Client.upload_model) method which takes the following arguments:

| Parameter | Type | Description |
|---|---|---|
| **path** | *String* (*Required*) | The file path to the model. |
| **framework** | *wallaroo.framework.Framework* (*Required*) | The model's framework.  See [Wallaroo SDK Essentials Guide: Model Uploads and Registrations](https://docs.wallaroo.ai/wallaroo-developer-guides/wallaroo-sdk-guides/wallaroo-sdk-essentials-guide/wallaroo-sdk-model-uploads/) for supported model frameworks. |
| **input_schema** | *pyarrow.lib.Schema* (*Optional*)  | The model's input schema.  **Only required for non-Native Wallaroo frameworks.  See [Wallaroo SDK Essentials Guide: Model Uploads and Registrations](https://docs.wallaroo.ai/wallaroo-developer-guides/wallaroo-sdk-guides/wallaroo-sdk-essentials-guide/wallaroo-sdk-model-uploads/) for more details. |
| **output_schema** | *pyarrow.lib.Schema* (*Optional*)  | The model's output schema.  **Only required for non-Native Wallaroo frameworks.  See [Wallaroo SDK Essentials Guide: Model Uploads and Registrations](https://docs.wallaroo.ai/wallaroo-developer-guides/wallaroo-sdk-guides/wallaroo-sdk-essentials-guide/wallaroo-sdk-model-uploads/) for more details. |
| **convert_wait** | *bool* (*Optional*)  | Whether to wait in the SDK session to complete the auto-packaging process for non-native Wallaroo frameworks. |
| **arch** | *wallaroo.engine_config.Architecture* (*Optional*)  | The targeted architecture for the model.  Options are <ol><li>`X86` (*Default*)</li><li>`ARM`</li></ol> |

We upload the model and set the architecture to `ARM`.

In [5]:
housing_model_control_arm = (wl.upload_model(model_name_arm, 
                                        model_file_name, 
                                        framework=Framework.ONNX,
                                        arch=Architecture.ARM)
                                        .configure(tensor_fields=["tensor"])
                        )

In [7]:
display(housing_model_control_arm)

Name,house-price-estimator-arm
Version,163ff0a9-0f1a-4229-bbf2-a19e4385f10f
File Name,rf_model.onnx
SHA,e22a0831aafd9917f3cc87a15ed267797f80e2afa12ad7d8810ca58f173b8cc6
Status,ready
Image Path,None
Architecture,arm
Acceleration,None
Updated At,2024-04-Mar 20:34:00


### Deploy Models to ARM Architecture

#### Build the Pipeline

This pipeline is used as an example of the edge deployment environment for testing.  We will create a pipeline and add our model set to the `ARM` architecture as a pipeline step.

The **model's targeted architecture** is inherited by the pipeline version; no additional pipeline settings are required to set that architecture.  The other settings apply the CPU and memory used by the model.

In [8]:
pipeline_arm = wl.build_pipeline(arm_pipeline_name)

# clear the previous steps
pipeline_arm.clear()

# set the model step with the ARM targeted model
pipeline_arm.add_model_step(housing_model_control_arm)

#### Deploy Pipeline

We deploy the pipeline, then show that the deployment configuration architecture is set to `ARM`.

In [12]:
#minimum deployment config
deploy_config = wallaroo.DeploymentConfigBuilder().replica_count(1).cpus(1).memory("1Gi").build()

pipeline_arm.deploy(pipeline_arm.deploy(deployment_config=deployment_config))

Waiting for deployment - this will take up to 45s .......... ok
Waiting for deployment - this will take up to 45s .............. ok


name,architecture-demonstration-arm
created,2024-03-04 20:34:08.895396+00:00
last_updated,2024-03-04 20:34:59.983185+00:00
deployed,True
arch,x86
accel,None
tags,
versions,"b72fb0db-e4b4-4936-a7cb-3d0fb7827a6f, 3ae70818-10f3-4f61-a998-dee5e2f00daf"
steps,house-price-estimator-arm
published,False


### Inferences on ARM Architecture Model Deployments

With the pipeline deployed, we'll perform sample inferences through the pipeline's inference API endpoints.  This will be the same method used for inference requests for model edge deployments.

In [13]:
# get the pipeline deployment urls

arm_deploy_url = pipeline_arm._deployment._url()

# get authorization header
headers = wl.auth.auth_header()

# set the content-type and accept headers

# set the content type for pandas records
headers['Content-Type']= 'application/json; format=pandas-records'

# set accept as pandas-records
headers['Accept']='application/json; format=pandas-records'

In [16]:
### inferences on arm model deployments

!curl -X POST {arm_deploy_url} \
    -H "Authorization: {headers['Authorization']}" \
        -H "Content-Type:{headers['Content-Type']}" \
            -H "Accept:{headers['Accept']}" \
                --data @./data/normal-inputs.df.json > ./arm_results.df.json

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  584k  100  475k  100  108k  27.3M  6394k --:--:-- --:--:-- --:--:-- 33.5M


In [19]:
# show the first 5 results from the outputs

arm_df = pd.read_json("./arm_results.df.json", orient="records")
arm_df.head(5)

,time,in,out,anomaly,metadata
0,1709584648784,"{'tensor': [4.0, 2.5, 2900.0, 5505.0, 2.0, 0.0, 0.0, 3.0, 8.0, 2900.0, 0.0, 47.6063, -122.02, 2970.0, 5251.0, 12.0, 0.0, 0.0]}",{'variable': [718013.7]},{'count': 0},"{'last_model': '{""model_name"":""house-price-estimator-arm"",""model_sha"":""e22a0831aafd9917f3cc87a15ed267797f80e2afa12ad7d8810ca58f173b8cc6""}', 'pipeline_version': 'b72fb0db-e4b4-4936-a7cb-3d0fb7827a6f', 'elapsed': [4090219, 3019440], 'dropped': [], 'partition': 'engine-5b544f9cbd-zkk5c'}"
1,1709584648784,"{'tensor': [2.0, 2.5, 2170.0, 6361.0, 1.0, 0.0, 2.0, 3.0, 8.0, 2170.0, 0.0, 47.7109, -122.017, 2310.0, 7419.0, 6.0, 0.0, 0.0]}",{'variable': [615094.6]},{'count': 0},"{'last_model': '{""model_name"":""house-price-estimator-arm"",""model_sha"":""e22a0831aafd9917f3cc87a15ed267797f80e2afa12ad7d8810ca58f173b8cc6""}', 'pipeline_version': 'b72fb0db-e4b4-4936-a7cb-3d0fb7827a6f', 'elapsed': [4090219, 3019440], 'dropped': [], 'partition': 'engine-5b544f9cbd-zkk5c'}"
2,1709584648784,"{'tensor': [3.0, 2.5, 1300.0, 812.0, 2.0, 0.0, 0.0, 3.0, 8.0, 880.0, 420.0, 47.5893, -122.317, 1300.0, 824.0, 6.0, 0.0, 0.0]}",{'variable': [448627.8]},{'count': 0},"{'last_model': '{""model_name"":""house-price-estimator-arm"",""model_sha"":""e22a0831aafd9917f3cc87a15ed267797f80e2afa12ad7d8810ca58f173b8cc6""}', 'pipeline_version': 'b72fb0db-e4b4-4936-a7cb-3d0fb7827a6f', 'elapsed': [4090219, 3019440], 'dropped': [], 'partition': 'engine-5b544f9cbd-zkk5c'}"
3,1709584648784,"{'tensor': [4.0, 2.5, 2500.0, 8540.0, 2.0, 0.0, 0.0, 3.0, 9.0, 2500.0, 0.0, 47.5759, -121.994, 2560.0, 8475.0, 24.0, 0.0, 0.0]}",{'variable': [758714.3]},{'count': 0},"{'last_model': '{""model_name"":""house-price-estimator-arm"",""model_sha"":""e22a0831aafd9917f3cc87a15ed267797f80e2afa12ad7d8810ca58f173b8cc6""}', 'pipeline_version': 'b72fb0db-e4b4-4936-a7cb-3d0fb7827a6f', 'elapsed': [4090219, 3019440], 'dropped': [], 'partition': 'engine-5b544f9cbd-zkk5c'}"
4,1709584648784,"{'tensor': [3.0, 1.75, 2200.0, 11520.0, 1.0, 0.0, 0.0, 4.0, 7.0, 2200.0, 0.0, 47.7659, -122.341, 1690.0, 8038.0, 62.0, 0.0, 0.0]}",{'variable': [513264.66]},{'count': 0},"{'last_model': '{""model_name"":""house-price-estimator-arm"",""model_sha"":""e22a0831aafd9917f3cc87a15ed267797f80e2afa12ad7d8810ca58f173b8cc6""}', 'pipeline_version': 'b72fb0db-e4b4-4936-a7cb-3d0fb7827a6f', 'elapsed': [4090219, 3019440], 'dropped': [], 'partition': 'engine-5b544f9cbd-zkk5c'}"


### Undeploy the Pipelines

With the inference examples complete, we can undeploy the pipelines.

In [25]:
pipeline_arm.undeploy()

Waiting for undeployment - this will take up to 45s .................................... ok
Waiting for undeployment - this will take up to 45s .................................... ok


name,architecture-demonstration-arm
created,2024-03-04 20:34:08.895396+00:00
last_updated,2024-03-04 20:45:55.098530+00:00
deployed,False
arch,x86
accel,None
tags,
versions,"74571863-9eb0-47aa-8b5a-3bdaa7aa9f03, b72fb0db-e4b4-4936-a7cb-3d0fb7827a6f, 3ae70818-10f3-4f61-a998-dee5e2f00daf"
steps,house-price-estimator-arm
published,False


## Edge Deployment

We now deploy the pipeline versions to our edge devices.

* Publish the pipeline versions:  Publishes the pipeline versions to the OCI registry with the deployment configuration inherited from the pipeline versions, with the architecture settings inherited from the model versions.
* Deploy Edge:  Deploy the edge device with the edge location settings.

### Publish Pipeline Versions

Publishing the pipeline uses the pipeline `wallaroo.pipeline.publish()` command.  This requires that the Wallaroo Ops instance have [Edge Registry Services](https://docs.wallaroo.ai/wallaroo-operations-guide/wallaroo-configuration/wallaroo-edge-deployment/#enable-wallaroo-edge-deployment-registry) enabled.

The following publishes the pipeline to the OCI registry and displays the container details.  For more information, see [Wallaroo SDK Essentials Guide: Pipeline Edge Publication](https://docs.wallaroo.ai/wallaroo-developer-guides/wallaroo-sdk-guides/wallaroo-sdk-essentials-guide/wallaroo-sdk-essentials-pipelines/wallaroo-sdk-essentials-pipeline-publication/).

In [22]:
assay_pub_arm = pipeline_arm.publish()
assay_pub_arm

Waiting for pipeline publish... It may take up to 600 sec.
Pipeline is Publishing.....Published.


ID,19
Pipeline Name,architecture-demonstration-arm
Pipeline Version,74571863-9eb0-47aa-8b5a-3bdaa7aa9f03
Status,Published
Engine URL,us-central1-docker.pkg.dev/wallaroo-dev-253816/uat/engines/proxy/wallaroo/ghcr.io/wallaroolabs/fitzroy-mini:v2024.1.0-main-4609
Pipeline URL,us-central1-docker.pkg.dev/wallaroo-dev-253816/uat/pipelines/architecture-demonstration-arm:74571863-9eb0-47aa-8b5a-3bdaa7aa9f03
Helm Chart URL,oci://us-central1-docker.pkg.dev/wallaroo-dev-253816/uat/charts/architecture-demonstration-arm
Helm Chart Reference,us-central1-docker.pkg.dev/wallaroo-dev-253816/uat/charts@sha256:ddd3ab601395437b8e8ffc45a6fddbbe564978cf6b07b6b0283debff137def36
Helm Chart Version,0.0.1-74571863-9eb0-47aa-8b5a-3bdaa7aa9f03
Engine Config,"{'engine': {'resources': {'limits': {'cpu': 4.0, 'memory': '3Gi'}, 'requests': {'cpu': 4.0, 'memory': '3Gi'}, 'accel': 'none', 'arch': 'x86', 'gpu': False}}, 'engineAux': {'autoscale': {'type': 'none'}, 'images': None}, 'enginelb': {'resources': {'limits': {'cpu': 1.0, 'memory': '512Mi'}, 'requests': {'cpu': 0.2, 'memory': '512Mi'}, 'accel': 'none', 'arch': 'x86', 'gpu': False}}}"
User Images,[]


### DevOps Deployment on ARM Architecture

The edge deployment is performed with `docker run`, `docker compose`, or `helm` installations.  The following command generates the `docker run` command, with the following values provided by the DevOps Engineer:

* `$REGISTRYURL`: The URL of the OCI registry service hosting the pipeline publish.
* `$OCI_USERNAME`: The username for the OCI registry service.
* `$OCI_PASSWORD`: The password or token for the OCI registry service.

For more details on model edge deployments with Wallaroo, see [Model Operations: Run Anywhere](https://staging.docs.wallaroo.ai/wallaroo-model-operations/wallaroo-model-operations-run-anywhere/).

In [24]:
# create docker run 

docker_command = f'''
docker run -p 8080:8080 \\
    -e DEBUG=true \\
    -e OCI_REGISTRY=$REGISTRYURL \\
    -e CONFIG_CPUS=1 \\
    -e OCI_USERNAME=$OCI_USERNAME \\
    -e OCI_PASSWORD=$OCI_PASSWORD \\
    -e PIPELINE_URL={assay_pub_arm.pipeline_url} \\
    {assay_pub_arm.engine_url}
'''

print(docker_command)


docker run -p 8080:8080 \
    -e DEBUG=true \
    -e OCI_REGISTRY=$REGISTRYURL \
    -e CONFIG_CPUS=1 \
    -e OCI_USERNAME=$OCI_USERNAME \
    -e OCI_PASSWORD=$OCI_PASSWORD \
    -e PIPELINE_URL=us-central1-docker.pkg.dev/wallaroo-dev-253816/uat/pipelines/architecture-demonstration-arm:74571863-9eb0-47aa-8b5a-3bdaa7aa9f03 \
    us-central1-docker.pkg.dev/wallaroo-dev-253816/uat/engines/proxy/wallaroo/ghcr.io/wallaroolabs/fitzroy-mini:v2024.1.0-main-4609



### Edge Inference Examples on ARM Architecture

The following examples demonstrate performing inferences on the model deployed on an `ARM` architecture edge device.  `HOSTNAME_ARM` is the hostname for the ARM architecture edge deployment.  Update this variable to match your deployment.

In [ ]:
HOSTNAME_ARM = 'localhost'

In [ ]:
# generate a set of normal house values from the ARM deployment
!curl -X POST {HOSTNAME_ARM}:8080/pipelines/{arm_pipeline_name} \
    -H "Content-Type: Content-Type: application/json; format=pandas-records" \
    --data @./data/normal-inputs.df.json > results.df.json

# display the first 5 results
result_df = pd.from_json('./results.df.json')
result_df.head(5)